In [51]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import (
    classification_report,
    precision_score,
    recall_score,
    accuracy_score,
    f1_score,
    r2_score,
)
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
from binning import bin

Read the train and test datasets


In [2]:
train_df = pd.read_csv("Training_set.csv")
test_df = pd.read_csv("Testing_set.csv")

Get the training features and label


In [3]:
X_train = train_df.iloc[:, :-1].to_numpy()
y_train = train_df.iloc[:, -1].to_numpy()

Get the testing features and label


In [4]:
X_test = test_df.iloc[:, :-1].to_numpy()
y_test = test_df.iloc[:, -1].to_numpy()

Initialize a 5-fold cross-validation object


In [40]:
kf = KFold(n_splits=5)

## MLP Classifier


Bin the label


In [5]:
y_train_binned = bin(y_train)
y_test_binned = bin(y_test)

Convert the label to categorical


In [41]:
y_train_binned_categorical = to_categorical(y_train_binned)

Iniatialize an MLP Classifier model


In [ ]:
mlp_class = Sequential()
mlp_class.add(Dense(256, activation="relu"))
mlp_class.add(Dense(len(np.unique(y_train_binned)), activation="softmax"))
mlp_class.compile(optimizer="adam", loss="categorical_crossentropy")

Define an EarlyStopping object


In [ ]:
early_stopping = EarlyStopping(patience=20)

Initialize scores lists


In [42]:
precisions = []
recalls = []
accuracies = []
f1_scores = []

Train the model using 5-fold cross-validation


In [ ]:
fold = 1
for train, test in kf.split(X_train, y_train_binned):
    print(f"##### FOLD: {fold} #####")

    # Fit the model
    mlp_class.fit(
        X_train[train],
        y_train_binned_categorical[train],
        epochs=100,
        batch_size=32,
        validation_data=(X_train[test], y_train_binned_categorical[test]),
        callbacks=[early_stopping],
    )

    # Predict on the test set
    predictions = np.argmax(mlp_class.predict(X_train[test]), axis=1)

    # Evaluate the model
    precision = precision_score(
        y_true=y_train_binned[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    recall = recall_score(
        y_true=y_train_binned[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    accuracy = accuracy_score(y_true=y_train_binned[test], y_pred=predictions)
    f1 = f1_score(
        y_true=y_train_binned[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )

    # Store the result
    precisions.append(precision)
    recalls.append(recall)
    accuracies.append(accuracy)
    f1_scores.append(f1)

    # Print the scores for each fold
    print(f"Precision = {precision}")
    print(f"Recall = {recall}")
    print(f"Accuracy = {accuracy}")
    print(f"F1 score = {f1}\n")

    fold += 1

Print the mean scores of the folds


In [ ]:
print("Mean Scores:")
print(f"Mean Precision = {np.mean(precisions)}")
print(f"Mean Recall = {np.mean(recalls)}")
print(f"Mean Accuracy = {np.mean(accuracies)}")
print(f"Mean F1 score = {np.mean(f1_scores)}")

Make predictions on the test set


In [ ]:
predictions = np.argmax(mlp_class.predict(X_test), axis=1)

Evaluate the model


In [ ]:
print(classification_report(y_true=y_test_binned, y_pred=predictions, zero_division=0))

## MLP Regressor


Iniatialize an MLP Regressor model


In [ ]:
mlp_reg = Sequential()
mlp_reg.add(Dense(256, activation="relu"))
mlp_reg.add(Dense(1))
mlp_reg.compile(optimizer="adam", loss="mse")

Define an EarlyStopping object


In [ ]:
early_stopping = EarlyStopping(patience=20)

Initialize scores lists


In [52]:
r2_scores = []

Train the model using 5-fold cross-validation


In [ ]:
fold = 1
for train, test in kf.split(X_train, y_train_binned):
    print(f"##### FOLD: {fold} #####")

    # Fit the model
    mlp_reg.fit(
        X_train[train],
        y_train[train],
        epochs=100,
        batch_size=32,
        validation_data=(X_train[test], y_train[test]),
        callbacks=[early_stopping],
    )

    # Predict on the test set
    predictions = mlp_reg.predict(X_train[test])

    # Evaluate the model
    r2 = r2_score(y_true=y_train[test], y_pred=predictions)

    # Store the result
    r2_scores.append(r2)

    # Print the scores for each fold
    print(f"R2 score = {r2}")

    fold += 1

Print the mean scores of the folds


In [ ]:
print(f"Mean R2 score = {np.mean(r2_scores)}")

Make predictions on the test set


In [54]:
predictions = mlp_reg.predict(X_test)

3012/3012 ━━━━━━━━━━━━━━━━━━━━ 2s 659us/step


Evaluate the model


In [55]:
print(r2_score(y_true=y_test, y_pred=predictions))

-0.0022766595525589572
